# Tree Metrics

### This notebook implements tree metrics for a specific dataset

##  Importing necessary libraries

Ensure that the Python environment you are running this in has all the libraries present in [requirements.txt](requirements.txt).


### Installing modules and libraries

In [ ]:
import typing
import json
import gdown

### Importing files

In [ ]:
!gdown <predicted-values>
!gdown <ground-truth-values>

In [ ]:
predicted_outputs = 'path_to_file'
ground_truth= 'path_to_file'

In [ ]:
with open(predicted_outputs, 'r') as file:
    pred = json.load(file)
with open(ground_truth, 'r') as file:
    tru = json.load(file)

### Implementation

In [ ]:
# Function generates a dependency graph and identifies free arguments from a sequence of API calls, 
# recording dependencies between calls based on argument references to previous calls ('PREV').

def build_graph(call_list: typing.List):
    """Return the dependency graph and free argument list for the APIs used in call_list.
    O(n^2)
    Args:
        call_list (typing.List): API call sequence as in the PS
    """
    # breakpoint()
    dependency_graph = {
        i['tool_name']: set() for i in call_list
    }

    free_arguments = {
        i['tool_name']: set() for i in call_list
    }

    for idx in range(len(call_list)-1, -1, -1):
        # print(idx)
        current_api = call_list[idx]
        for argument in current_api["arguments"]:
            # debug(argument)
            name = argument["argument_name"]
            if isinstance(argument["argument_value"], str):
              val = argument["argument_value"]
            elif isinstance(argument["argument_value"],list):
              if(argument["argument_value"] != []):
                val = argument["argument_value"][0]
              else:
                val = ""
            else:
              val = ""

            if 'PREV' in val:
                prev_idx = int(val.split('[')[1].split(']')[0])
                # debug(prev_idx)
                dependency_graph[current_api["tool_name"]].add(
                    (name, call_list[prev_idx]["tool_name"]))
            else:
                # debug(name)
                free_arguments[current_api["tool_name"]].add((name, val))
        # debug(dependency_graph)
        # debug(free_arguments)
    return dependency_graph, free_arguments

In [ ]:
# Function compares if two graphs, represented as dictionaries, are structurally identical, returning True if they are, and False otherwise.

def check_isomorphic(graph_1: typing.Dict, graph_2: typing.Dict):
    if len(graph_1) != len(graph_2):
        return False
    for arg in graph_1:
        if arg in graph_2:
            if graph_1[arg] != graph_2[arg]:
                return False
        else:
            return False
    return True

In [ ]:
# Function compares two sequences of API calls, returning True if they are structurally identical, and False otherwise.

def compare(call_list_1: typing.List, call_list_2: typing.List, mode: str = "ABS"):
    dep, arg = build_graph(call_list_1)
    dep_nxt, arg_nxt = build_graph(call_list_2)
    if mode == "ABS":
        return check_isomorphic(dep, dep_nxt) and check_isomorphic(arg, arg_nxt)
    else:
        return (int(check_isomorphic(dep, dep_nxt)) + int(check_isomorphic(arg, arg_nxt)))/2

In [ ]:
# Debug function

DEBUG = True

def debug(string):
    if DEBUG:
        print(string)

### Execution

In [ ]:
for i in range(20):
  DEBUG = False
  dep,  arg = build_graph(pred[0])
  dep_nxt, arg_nxt = build_graph(tru[0])

  print(compare(pred[i], tru[i]))